In [1]:
from keras.optimizers import Adam
from keras.models import Model
from keras import backend as K
from datetime import datetime
import distutils.dir_util
from keras.callbacks import CSVLogger
from keras.utils.np_utils import to_categorical
import numpy as np

from keras.layers import Dense, Input, Lambda, BatchNormalization, Conv2D, Dropout
from keras.layers import ZeroPadding2D, MaxPooling2D, AveragePooling2D, Activation, Flatten
from keras.layers import GlobalAveragePooling2D
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np

from keras import __version__
print(__version__)

Using TensorFlow backend.


2.0.3


In [2]:
base_model = VGG16(weights='imagenet', include_top=False)

In [3]:
# get data for visualisation later

from keras.preprocessing.image import ImageDataGenerator
# def preprocess_input(x, data_format=None):
#     """Preprocesses a tensor encoding a batch of images.
#     # Arguments
#         x: input Numpy tensor, 4D.
#         data_format: data format of the image tensor.
#     # Returns
#         Preprocessed tensor.
#     """

#     # 'RGB'->'BGR'
#     x = x[:, :, ::-1]
#     # Zero-center by mean pixel
#     x[:, :, 0] -= 103.939
#     x[:, :, 1] -= 116.779
#     x[:, :, 2] -= 123.68
#     return x

train_datagen = ImageDataGenerator(rescale=1./255)

validation_datagen = ImageDataGenerator(rescale=1./255)

#batch_size = 32

train_generator = train_datagen.flow_from_directory(
        '/home/ubuntu/data/fishing/train',
        target_size=(360, 640),
        batch_size=28,
        #batch_size=batch_size,
        shuffle=False,
        class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
        '/home/ubuntu/data/fishing/valid',
        target_size=(360, 640),
        shuffle=False,
        #batch_size=batch_size,
        batch_size=10,
        class_mode='categorical')

test_generator = validation_datagen.flow_from_directory(
        '/home/ubuntu/data/fishing/test_stg1',
        target_size=(360, 640),
        shuffle=False,
        #batch_size=batch_size,
        batch_size=20,
        class_mode='categorical')

Found 3276 images belonging to 8 classes.
Found 500 images belonging to 8 classes.
Found 1000 images belonging to 1 classes.


In [4]:
def create_precomputed_data(model, batches):
    filenames = batches.filenames
    conv_features = model.predict_generator(batches, (batches.samples/batches.batch_size ), verbose=1)
    labels_onehot = to_categorical(batches.classes)
    labels = batches.classes
    return (filenames, conv_features, labels_onehot, labels)

In [5]:
trn_filenames, trn_conv_features, trn_labels, trn_labels_1 = create_precomputed_data(base_model, train_generator)
val_filenames, val_conv_features, val_labels, val_labels_1 = create_precomputed_data(base_model, validation_generator)
test_filenames, test_conv_features, test_labels, test_labels_1 = create_precomputed_data(base_model, test_generator)


50/50 [==============================] - 110s   


In [7]:
RESULTS_DIR = '/home/ubuntu/data/fishing/results'

import bcolz
def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()

def save_precomputed_data(filenames, conv_feats, labels, features_base_name="VGG16_640_conv_feats/trn_"):
    save_array(RESULTS_DIR+"/"+features_base_name+'filenames.dat', np.array(filenames))
    save_array(RESULTS_DIR+"/"+features_base_name+'conv_feats.dat', conv_feats)
    save_array(RESULTS_DIR+"/"+features_base_name+'labels.dat', np.array(labels))
    
save_precomputed_data(trn_filenames, trn_conv_features, trn_labels, "VGG16_640_conv_feats/trn_")
save_precomputed_data(val_filenames, val_conv_features, val_labels, "VGG16_640_conv_feats/val_")
save_precomputed_data(test_filenames, test_conv_features, test_labels, "VGG16_640_conv_feats/test_")

In [8]:
import bcolz
def load_array(fname):
    return bcolz.open(fname)[:]

def load_precomputed_data(features_base_name="VGG16_640_conv_feats/trn_"):
    filenames = load_array(RESULTS_DIR+"/"+features_base_name+'filenames.dat').tolist()
    conv_feats = load_array(RESULTS_DIR+"/"+features_base_name+'conv_feats.dat')
    labels = load_array(RESULTS_DIR+"/"+features_base_name+'labels.dat')
    return filenames, conv_feats, labels

trn_filenames, trn_conv_features, trn_labels = load_precomputed_data("VGG16_640_conv_feats/trn_")
val_filenames, val_conv_features, val_labels = load_precomputed_data("VGG16_640_conv_feats/val_")
test_filenames, test_conv_features, test_labels = load_precomputed_data("VGG16_640_conv_feats/test_")